<a href="https://colab.research.google.com/github/shaimaaosman/ESP_Training/blob/master/Cnn_pr0ject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install livelossplot


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#importting the keras libirary and packages
import pickle
import gzip
import os
import random
from livelossplot import PlotLossesKeras
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from tensorflow.keras.layers import Dense, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import  Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive
np.random.seed(np.random.randint(0, 40))
#print number of version from Tensorflow is useing 
print('Tensorflow version:', tf.__version__)


In [ ]:

'''loads a saved pickle file that represents an object and returns that object'''
def load(file_name):
    # load the model
    stream = gzip.open(file_name, "rb")
    model = pickle.load(stream)
    stream.close()
    return model
'''save model that is an object as file_name as a pickle file'''
def save(file_name, model):
    # save the model
    stream = gzip.open(file_name, "wb")
    pickle.dump(model, stream)
    stream.close()


In [ ]:
#preprocessing
from google.colab import drive
train_datagen = ImageDataGenerator(
                #samplewise_std_normalization = True,
                rescale = 1./255,
                zoom_range=0.2,
                validation_split = 0.30,
                horizontal_flip = True,
                vertical_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/dataset',
        target_size=(256, 256),
        batch_size=98,
        class_mode='categorical',
        subset ='training')
validation_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/dataset',
        target_size=(256, 256),
        batch_size=98,
        class_mode='categorical',
        subset ='validation')


test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/dataset',
        target_size=(256, 256),
        batch_size=100,
        class_mode='categorical')




In [ ]:

'''The CNN Model'''

# Initialising the CNN
model = Sequential()

# 1st Convolution
model.add(Conv2D(32,(5,5), padding='same', input_shape=(256, 256,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(64,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(6, activation='softmax'))



In [ ]:
'''sheduling the learning rate and compiling the model'''

initial_learning_rate = 0.005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=5,
    decay_rate=0.96,
    staircase=True)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
'''Training the model'''

#using early stopping to exit training if validation loss is not decreasing even after certain epochs  
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

#save the bast model with lower validation loss
checkpoint = ModelCheckpoint("{epoch}th_epoch_model_weights.h5", monitor='val_loss',
                              save_weights_only=True, mode='min', verbose=1,  save_best_only=True)


callbacks = [PlotLossesKeras()]#, reduce_lr]

history = model.fit_generator(
    train_generator,
    steps_per_epoch= train_generator.n // 98,
    epochs = 37,
    validation_data= validation_generator,
    validation_steps= validation_generator.n // 98,
    callbacks=callbacks)


'''Evaluating the model'''

evaluate = model.evaluate_generator(test_generator, steps = test_generator.n // 100, verbose =1)
classes = ['beach', 'desert', 'forest', 'industrial', 'mountain', 'sea ice']
fig, axs = plt.subplots(5,6, figsize=(32,32))
main_dir = '/content/drive/My Drive/dataset'
for i in range(5) :
    cnt = 0
    for dir in os.listdir(main_dir) :
        images = os.listdir(os.path.join(main_dir, dir))
        j = random.randint(0, len(images))
        image = Image.open(os.path.join(main_dir, dir, images[j]))
        image = np.asarray(image, dtype = 'float64')
        image  = image / 255.
        y_pred = np.argmax(model.predict(np.asarray([image])), 1)
        axs[i][cnt].imshow(image)
        axs[i][cnt].set_title("prediction : {} , true class : {}".format(classes[y_pred[0]], dir), fontsize = 16)
        cnt = cnt + 1
fig.tight_layout()
    
model.save_weights('mymodel_weights.h5')
model.save('mymodel.h5')

In [ ]:
model.load_weights('mymodel_weights.h5')

evaluate = model.evaluate_generator(test_generator, steps = test_generator.n // 100, verbose =1)
